# Demo MultiContainer with ShiftAmount Activity
In certain scenarios it is important to distinguish which types of objects are available in a container. HasContainer does not provide this capability, while MultiContainer do. A MultiContainer represents a named set of containers. The number of allowed containers is limited by parameter **store_capacity**.
For each container a name (**id_** property), a **capacity** and **level** is specified. 
A MultiContainer can be initialized with parameter **initials** as provided in the example for site and vessel configuration below.

In [1]:
import datetime, time
import simpy

import shapely.geometry

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

In [2]:
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasMultiContainer,
        core.HasResource,
    ),
    {},
)

TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.MultiContainerDependentMovable,
        core.Processor,
        core.HasResource,
        core.LoadingFunction,
        core.UnloadingFunction,
    ),
    {},
)

location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)

## Definition of Site with MultiContainer
The clas HasContainer has now been replaced with HasMultiContainer. The **from_site** can contain four different container types, but right now contains only two: One called MP with a capacity of 10 and a level of 2 and one called TP with a capacity of 2 and a level of 0.

In [3]:
from_site = Site(
    env=my_env,
    name="Winlocatie",
    ID="6dbbbdf4-4589-11e9-a501-b469212bff5d",
    geometry=location_from_site,
    store_capacity=4,
    initials=[
        {"id": "MP", "level": 2, "capacity": 10},
        {"id": "TP", "level": 0, "capacity": 2},
    ],
)

## Creation of Vessel with MultiContainer
The class ContainerDependentMovable has been replaced with MultiContainerDependentMovable. The vessel has two containers, one for MPs and one for TPs, each with a capacity of two and a current level of zero.

In [4]:
hopper = TransportProcessingResource(
    env=my_env,
    name="Hopper 01",
    ID="6dbbbdf6-4589-11e9-95a2-b469212bff5b",
    geometry=location_from_site,
    loading_rate=1,
    unloading_rate=1,
    store_capacity=4,
    compute_v=lambda x: 10,
    initials=[
        {"id": "MP", "level": 0, "capacity": 2},
        {"id": "TP", "level": 0, "capacity": 2},
    ],
)

## ShiftAmount Activity for MultiContainer
The **amount** specifies the objects to be transfered and the **id_** parameter specifies to which container this activity relates.

In [5]:
activity = model.ShiftAmountActivity(
    env=my_env,
    name="Transfer MP",
    ID="6dbbbdf7-4589-11e9-bf3b-b469212bff52",
    registry=registry,
    processor=hopper,
    origin=from_site,
    destination=hopper,
    amount=1,
    id_="MP",
    duration=20,
)

## Run simulation

In [6]:
model.register_processes([activity])
my_env.run()

In [7]:
display(plot.get_log_dataframe(hopper, [activity]))
display(plot.get_log_dataframe(activity, [activity]))

,Activity,Timestamp,ActivityState,geometry,container level
0,Transfer MP,1970-01-01 00:00:00,START,POINT (4.18055556 52.18664444),"{'MP': 0, 'TP': 0}"
1,Transfer MP,1970-01-01 00:00:00,START,POINT (4.18055556 52.18664444),"{'MP': 0, 'TP': 0}"
2,Transfer MP,1970-01-01 00:00:20,STOP,POINT (4.18055556 52.18664444),"{'TP': 0, 'MP': 1}"
3,Transfer MP,1970-01-01 00:00:20,STOP,POINT (4.18055556 52.18664444),"{'TP': 0, 'MP': 1}"


,Activity,Timestamp,ActivityState
0,Transfer MP,1970-01-01 00:00:00,START
1,Transfer MP,1970-01-01 00:00:20,STOP
